In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.io as sio

In [19]:
mat_contents1 = sio.loadmat('Condition1.mat')
mat1 = mat_contents1['final_mat_condition3']
# print(type(mat_contents1)) 
# mat_contents1.items()
mat1.shape

(40, 769)

In [20]:
mat_contents2 = sio.loadmat('Condition2.mat')
mat2 = mat_contents2['final_mat_condition7']
# print(type(mat_contents2)) 
# mat_contents2.items()
mat2.shape

(40, 769)